This notebook will be used for the Capstone Project for IBM's Professional Certificate

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


Week 3 Assignment

In [3]:
!pip install requests
!pip install html5lib
!pip install bs4
!pip install geocoder

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

##Website to scrape below
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
##Convert to text
soup = BeautifulSoup(website_text,'xml')


table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

#creating a variable for BeautifulSoup to send data to 
data = []

#Splitting and stripping the table apart where it finds 'td' in the HTML code
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

##labeling index in dataframe    
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])

## Deleting Blank Entries in table
df = df[~df['PostalCode'].isnull()]  

## Deleting "Not Assigned" Boroughs
df = df[~df.Borough.str.contains("Not assigned")]

##Filter for changing "Not Assigned" Neighborhoods to their Borough name if possible
filter1 = df['Neighborhood']
for x in filter1:
    if x == "Not assigned":
        df['Borough'] = df['Borough'].replace([2], [3])
    else:
        df['Borough'] = df['Borough'].replace([2], [2])
        
## Wikipedia has already combined the Boroughs but I left this here to show how I would group it back together        
df.groupby("Borough")['Neighborhood'].apply(lambda Neighbourhood: ','.join(Neighbourhood))     

df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
print(df.shape)

(103, 3)


In [6]:
links={'Postal Code':'https://cocl.us/Geospatial_data',\
       'Latitude':'https://cocl.us/Geospatial_data',\
       'Longitude':'https://cocl.us/Geospatial_data'}
df_geodata = pd.read_csv(links['Postal Code'])
df_geodata.columns = ['PostalCode', 'Latitude','Longitude']
df_geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
latlist = []
longlist = []
for PC in df['PostalCode']:
    lat = df_geodata.loc[df_geodata.PostalCode == PC, 'Latitude'].values[0]
    long = df_geodata.loc[df_geodata.PostalCode == PC, 'Longitude'].values[0]
    latlist.append(lat)
    longlist.append(long)
   
df['Latitude'] = latlist
df['Longitude'] = longlist
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
dfToronto = df[df['Borough'].str.contains("Toronto")]
dfToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
23,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
31,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


 

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [10]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:

CLIENT_ID = 'K4AVNJ02ZGXDGFLY4X5WCKYCNUKVP05H51POSJ3KPYM41D5X' # your Foursquare ID
CLIENT_SECRET = '2DOOHOHCOXGXZHWBCH5N5O5NIB1LVC11KPUF4CMFOGUO5H3O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = dfToronto.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Your credentails:
CLIENT_ID: K4AVNJ02ZGXDGFLY4X5WCKYCNUKVP05H51POSJ3KPYM41D5X
CLIENT_SECRET:2DOOHOHCOXGXZHWBCH5N5O5NIB1LVC11KPUF4CMFOGUO5H3O
Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )
Toronto_venues.groupby('Neighborhood').count()

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-71]] + list(Toronto_onehot.columns[:-71])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [13]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)


    

In [14]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = dfToronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [15]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,0,Coffee Shop,Bakery,Café,Breakfast Spot,Hotel,Distribution Center,Health Food Store,Gym / Fitness Center,Brewery,Cosmetics Shop
7,Downtown Toronto,0,Coffee Shop,Music Venue,Bank,Distribution Center,Diner,Fried Chicken Joint,Creperie,College Cafeteria,College Auditorium,Gym
14,Downtown Toronto,0,Clothing Store,Coffee Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Bookstore
23,Downtown Toronto,0,Café,Coffee Shop,Gastropub,American Restaurant,Cocktail Bar,Creperie,Cosmetics Shop,Department Store,Lingerie Store,Hotel
32,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Cheese Shop,Concert Hall,Bistro,Juice Bar,Jazz Club
41,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Korean Restaurant,Indian Restaurant,Ice Cream Shop,Gym / Fitness Center
50,Downtown Toronto,0,Coffee Shop,Café,Gym,Deli / Bodega,Clothing Store,Bakery,Burrito Place,Bookstore,Cosmetics Shop,Hotel
59,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Brewery,Italian Restaurant,Music Venue,Baseball Stadium,Bar
67,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Frozen Yogurt Shop,Bookstore,Cosmetics Shop,Juice Bar,Indian Restaurant
68,Downtown Toronto,0,Coffee Shop,Hotel,Café,Bakery,Japanese Restaurant,Italian Restaurant,American Restaurant,Beer Bar,Gastropub,Breakfast Spot


In [16]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Central Toronto,1,Garden,Fast Food Restaurant,Adult Boutique,German Restaurant,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel


In [17]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Downtown Toronto,2,Grocery Store,Café,Coffee Shop,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,German Restaurant,Gastropub,Gay Bar


In [18]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
140,Downtown Toronto,3,Airport Service,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Boat or Ferry,Boutique,Adult Boutique


In [19]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,East Toronto,4,Health Food Store,Adult Boutique,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel
51,West Toronto,4,Bakery,Music Venue,Middle Eastern Restaurant,Grocery Store,Brewery,Bar,Bank,Café,Flea Market,Fish Market
60,West Toronto,4,Bar,Café,Asian Restaurant,Men's Store,Japanese Restaurant,Cuban Restaurant,Korean Restaurant,Beer Store,Italian Restaurant,Diner
76,East Toronto,4,Fast Food Restaurant,Ice Cream Shop,Gym,Movie Theater,Italian Restaurant,Light Rail Station,Fish & Chips Shop,Brewery,Liquor Store,Food & Drink Shop
94,Central Toronto,4,Business Service,Bus Line,Ethiopian Restaurant,Gluten-free Restaurant,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant
103,Central Toronto,4,Food & Drink Shop,Hotel,Breakfast Spot,Department Store,Dance Studio,Adult Boutique,Gas Station,Gastropub,Gay Bar,General Entertainment
104,Central Toronto,4,Jewelry Store,Adult Boutique,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel
105,West Toronto,4,Café,Mexican Restaurant,Music Venue,Discount Store,Cajun / Creole Restaurant,Grocery Store,Italian Restaurant,Bookstore,Gastropub,Bar
122,Downtown Toronto,4,Café,Bar,Bakery,Bookstore,Japanese Restaurant,French Restaurant,Comfort Food Restaurant,Bank,Italian Restaurant,Beer Bar
130,Central Toronto,4,Adult Boutique,Health Food Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel
